In [1]:
# imports
import pandas as pd
import numpy as np
!pip install gurobipy  # install gurobipy, if not already installed
import gurobipy as grb  # import the installed package
from gurobipy import GRB

In [2]:
datafile = 'GeorgiaCountyData2.csv'
data = pd.read_csv(datafile)
data.head()

,County,Democratic Votes,Republican Votes,Total Votes,Population
0,Appling,"1,784","6,570","8,390","18,428"
1,Atkinson,825,"2,300","3,155","8,311"
2,Bacon,625,"4,017","4,667","11,140"
3,Baker,652,897,"1,555","3,090"
4,Baldwin,"9,140","8,903","18,261","45,072"


In [3]:
data['Democratic Votes'] = data['Democratic Votes'].str.replace(',', '').astype(float)
data['Republican Votes'] = data['Republican Votes'].str.replace(',', '').astype(float)
data['Total Votes'] = data['Total Votes'].str.replace(',', '').astype(float)
data['Population'] = data['Population'].str.replace(',', '').astype(float)

The vote-shares are normalized so that the population in each district is exactly 1; for a state with n districts, the vote-shares s1 and s2 for Player 1 and Player 2, respectively, satisfy s1 + s2 = n

In [4]:
# normalize population and vote shares ^ see above block
data['Scaled Population'] = np.array(data['Population'])/(np.sum(data['Population'])/14)
data['Dem % Votes'] = data['Democratic Votes']/(data['Democratic Votes'] + data['Republican Votes'])
data['Dem Seats'] = data['Dem % Votes'] * data['Scaled Population']
data['Rep % Votes'] = data['Republican Votes']/(data['Democratic Votes'] + data['Republican Votes'])
data['Rep Seats'] = data['Rep % Votes'] * data['Scaled Population']
#data

In [5]:
datafile2 = 'county_adjacency2010.csv'
data2 = pd.read_csv(datafile2)
s1 =data2['fipscounty']>=13001
s2 = data2['fipscounty']<=13321
s3 = s1*s2
df2 = pd.DataFrame()
df2['countyname'] = np.where(s3,data2['countyname'],0)
df2['fipscounty'] = np.where(s3,data2['fipscounty'],0)
df2['fipsneighbor'] = np.where(s3,data2['fipsneighbor'],0)
df2 = df2.loc[~(df2==0).all(axis=1)]
names = []
for i in df2['countyname']:
  name = i
  n = np.char.split([name], sep = " ")[0][0]
  names.append(n)
names
df2['Name'] = names
fipCodes = df2['fipscounty'].unique()
GeorgiaNeighbors = []
for i in fipCodes:
  neighbors = np.where(df2['fipscounty']==i, df2['fipsneighbor'],0)
  neighbors = neighbors[neighbors!=0]
  GeorgiaNeighbors.append([i,[neighbors]])
fipdf = pd.DataFrame()
n = np.append(np.unique(names),['Fulton','Fulton','Gwinnett','Gwinnett','Cobb','Cobb', 'DeKalb','DeKalb'])
fipdf['Name'] = n
fipCodes = np.append(fipCodes,[13121,13121,13135,13135,13067,13067,13087,13087] )
fipdf['Fip'] = fipCodes
fipdf = fipdf.sort_values(by=['Name'],ignore_index=True)
data['fip'] = np.where(data['County']==fipdf['Name'],fipdf['Fip'],0)

In [6]:
f = np.where(data['County']=='DeKalb',13089,data['fip'])
f = np.where(data['County']=='Ben Hill',13017,f)
f = np.where(data['County']=='Decatur',13087,f)
f = np.where(data['County']=='Jeff Davis',13161,f)
data['fip'] = f
#fixing those that were missed
data = data.reset_index().rename(columns = {'index': 'county_id'})
data.head()
data

,county_id,County,Democratic Votes,Republican Votes,Total Votes,Population,Scaled Population,Dem % Votes,Dem Seats,Rep % Votes,Rep Seats,fip
0,0,Appling,1784.0,6570.0,8390.0,18428.0,0.024532,0.213550,0.005239,0.786450,0.019293,13001
1,1,Atkinson,825.0,2300.0,3155.0,8311.0,0.011064,0.264000,0.002921,0.736000,0.008143,13003
2,2,Bacon,625.0,4017.0,4667.0,11140.0,0.014830,0.134640,0.001997,0.865360,0.012833,13005
3,3,Baker,652.0,897.0,1555.0,3090.0,0.004114,0.420917,0.001731,0.579083,0.002382,13007
4,4,Baldwin,9140.0,8903.0,18261.0,45072.0,0.060001,0.506568,0.030395,0.493432,0.029607,13009
...,...,...,...,...,...,...,...,...,...,...,...,...
162,162,Whitfield,10680.0,25644.0,36766.0,104122.0,0.138610,0.294020,0.040754,0.705980,0.097856,13313
163,163,Wilcox,861.0,2402.0,3279.0,8701.0,0.011583,0.263868,0.003056,0.736132,0.008527,13315
164,164,Wilkes,2160.0,2823.0,5031.0,9797.0,0.013042,0.433474,0.005653,0.566526,0.007389,13317
165,165,Wilkinson,2074.0,2665.0,4770.0,8945.0,0.011908,0.437645,0.005211,0.562355,0.006696,13319


In [7]:
# read in locational data
cij_df = pd.read_csv('cij.csv').drop(columns = ['Unnamed: 0'])
cij_df.head()

,index_i,county_i,index_j,county_j,cij
0,0,Appling,0,Appling,0
1,0,Appling,1,Atkinson,0
2,0,Appling,2,Bacon,1
3,0,Appling,3,Baker,0
4,0,Appling,4,Baldwin,0


In [8]:
cij_matrix = np.zeros((len(fipdf),len(fipdf)))
for i in range(len(fipdf)):
  for j in range(len(fipdf)):
    cij_matrix[i, j] = cij_df[(cij_df.index_i == i) & (cij_df.index_j == j)]['cij'].iloc[0]

cij_matrix

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [9]:
#In the continuous nongeometric setting of the bisection protocol, the vote-share thresholds tm,j
#needed to win j of m districts are computed from the recurrence in Lemma 2


def lemma2(n,j):
  if j ==0:
    return 0,0
  elif j==1 and n==1:
    return 1/2,0
  else:
    a = int(np.floor(n/2))
    b = int(np.ceil(n/2))
    Max = np.minimum(j,a)
    Min = np.maximum(0,j-b)
    K = np.arange(Min,Max+1) 
    t = 0
    T = []
    kStar = 0
    for k in K:
      x = n-lemma2(a,a-k+1)[0] - lemma2(b,b-(j-k)+1)[0]
      T.append(x)
    if len(T) ==1:
      kStar = k
      return T[0],kStar
    elif len(T)>1:
      t = min(T)
      kStar = k
    return t,kStar 

In [10]:
def jandkstar (m,vs): #uses lemma2
  J = np.arange(1,m+1)
  #print(J)
  Jval = []
  jstar = 0
  kstar = 0
  mi = 0.0
  for j in J:
    t = float(lemma2(m,j)[0])
    k = (lemma2(m,j)[1])
    Jval.append(t)
    if t <= vs and t>mi:
      mi = t
      jstar = j
      kstar = k
  return jstar,kstar 

In [11]:
S = []
S.append(data['county_id'])
S.append(data['Scaled Population'])
S.append(data['Dem Seats'])
S.append(data['Rep Seats'])

In [12]:
T = .03

In [13]:
#did not end up using
def contig(A):
  for a in A: #assuming it is a list of fips
    l = list(data['fip']== a)
    ind = l.index(True)
    neigh = GeorgiaNeighbors[ind][1][0]
    flag = False
    for b in A:
      if b!=a:
        if b in neigh:
          flag = True
    if flag == False:
      return flag
  return flag

In [14]:
# MIP program

def Bisection(S0,M0,j_star,k_star,d):
    #f = S[3]
    counties = S0[0]
    
    p0 = S0[1] #population vector for each county
    a = np.floor(M0/2) #smaller set size
    if a<1:
        a=1
    b = np.ceil(np.round(M0,0)/2)
    #print(M0)
    #print(a,b)
    tak = lemma2(a,k_star)[0]
    tbjk = lemma2(b,j_star-k_star)[0]
    Pa = a/M0 * sum(p0) #really should be the sum of all population...
    T = .05
    if d == 0:
        vd = (S0[2])
        Vnot = (S0[3]) #voteshare for other player in S
    else:
        vd = (S0[3])
        Vnot = (S0[2])
    j = 0
    m = grb.Model('BIS')
  #  variables
    x = {}
      # Creating variables
    for j in range(len(p0)):
        x[j] = m.addVar(vtype=GRB.BINARY) #(0,1) is county is in set A

    J = len(p0)
  # Objective function
    
    m.setObjective(grb.quicksum( -x[i] * x[j] * cij_matrix[counties[i], counties[j]] for i, j in zip(range(J), range(J))) + grb.quicksum( (1-x[i]) * (1-x[j]) * cij_matrix[counties[i], counties[j]] for i, j in zip(range(J), range(J)))) #arbitrary

  # Generating constraints
    m.addConstr(grb.quicksum(x[j]*p0[j] for j in range(J)) >= (1-T)*Pa )
    m.addConstr( grb.quicksum(x[j]*p0[j] for j in range(J)) <= (1+T)*Pa )

    
    m.addConstr( grb.quicksum(x[j]*vd[j] for j in range(J)) >= (tak/a)*grb.quicksum(x[j]*(vd[j]+Vnot[j]) for j in range(J)))
    m.addConstr( grb.quicksum((1-x[j])*vd[j] for j in range(J)) >= (tbjk/b)*grb.quicksum((1-x[j])*(vd[j]+Vnot[j]) for j in range(J)))

    #locational constraint to be added
    #can feed in fip codes. can get list of fips in A set. 
    
    #m.addConstr( grb.quicksum( x[i] * x[j] * cij_matrix[counties[i], counties[j]] for i, j in zip(range(J), range(J))) >= grb.quicksum(2*x[j] for j in range(J)) -2)
    #m.addConstr( grb.quicksum( (1-x[i]) * (1-x[j]) * cij_matrix[counties[i], counties[j]] for i, j in zip(range(J), range(J))) >= 2*(grb.quicksum((1-x[j])) for j in range(J)) -1 )    
    #m.addConstr( grb.quicksum( (1-x[i]) * (1-x[i]) * cij_matrix[counties[i], counties[j]] for i, j in zip(range(J), range(J))) >= grb.quicksum(2*(1-x[i]) for j in range(J)) -2)
  #m.addConstr( grb.quicksum( cij_matrix[counties[i], counties[j]] for i, j in zip(range(J), range(J))) >= 2*(grb.quicksum(x[j]) for j in range(J)) -1 )
  #  m.addConstr( grb.quicksum( cij_matrix[counties[i], counties[j]] for i, j in zip(range(J), range(J))) >= 2*(grb.quicksum((1-x[j])) for j in range(J)) -1 )    

    m.setParam( 'OutputFlag', False )
    m.optimize()
    # Check to see what happened!
    #if (m.status != grb.GRB.OPTIMAL):
        #print('not optimal')

    return m


In [15]:
import time
a = time.time()
def bisect(S0, M0, d):
    j_star,k_star= jandkstar (M0,sum(S[1]))
    #print(j_star,k_star)
    a = time.time()
    m0 = Bisection(S0,M0,j_star,k_star,d)
    b = time.time()
    print('{} seconds for original bisection'.format(b-a))
    while m0.status != grb.GRB.OPTIMAL: 
        print('running again...')
        #print(j_star)
    #Bisection(S,m,j∗,k∗,d) is infeasible
        j_star = j_star - 1
        if j_star == -1:
          print("Infeasible")
          return ""
    # recompute k*
        k_star = lemma2(M0,j_star)[1]
        #print(j_star,k_star)
        a = time.time()
        m0 = Bisection(S0,M0,j_star,k_star,d)
        b = time.time()
        print('{} seconds to run again'.format(b-a))
    #print(j_star)
    return Bisection(S0,M0,j_star,k_star,d)

## Current version is if the Republican player starts

In [ ]:
A = bisect(S, 14, 1).X #player 1 starting
A = np.array(A)
a1 = A
b1 = (1-A)
A

In [ ]:
a1f = np.array(a1*data.index)
#this misses the 0 index

if a1[0] == 1:
    a1f[0] = 1
    a1f = a1f[a1f!=0]
    a1f[0] = 0
else:
    a1f = a1f[a1f!=0]

adf = data[data.index.isin(a1f)]
adf = adf.reset_index(drop=True)


b1f = np.array(b1*data.index)

if b1[0] == 1:
    b1f[0] = 1
    b1f = b1f[b1f!=0]
    b1f[0] = 0
else:
    b1f = b1f[b1f!=0]

bdf = data[data.index.isin(b1f)]
bdf = bdf.reset_index(drop=True)


In [ ]:
#for visualization later:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
import plotly.express as px

In [ ]:
fig = px.choropleth_mapbox(adf, geojson=counties, locations='fip',
                           color_continuous_scale=px.colors.sequential.Jet,
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'District':'District'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.choropleth_mapbox(bdf, geojson=counties, locations='fip',
                           color_continuous_scale=px.colors.sequential.Jet,
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'District':'District'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
S1 = []
S1.append(adf['county_id'])
S1.append(adf['Scaled Population'])
S1.append(adf['Dem Seats'])
S1.append(adf['Rep Seats'])

#M is number of districts for each piece.
M1 = sum(S1[1]) + sum(S1[2]) 

A1 = bisect(S1, 7, 0).X #player 2 bisects both

S2 = []
S2.append(bdf['county_id'])
S2.append(bdf['Scaled Population'])
S2.append(bdf['Dem Seats'])
S2.append(bdf['Rep Seats'])

#M is number of districts for each piece.
M2 = sum(S2[1]) + sum(S2[2]) 
A2 = bisect(S2, 7, 0).X

In [ ]:
#side A

A1 = np.array(A1)
a1 = A1
b1 = (1-A1)
a1f = np.array(a1*adf.index)

if a1[0] == 1:
    a1f[0] = 1
    a1f = a1f[a1f!=0]
    a1f[0] = 0
else:
    a1f = a1f[a1f!=0]

adf2 = adf[adf.index.isin(a1f)]
adf2 = adf2.reset_index(drop=True)


b1f = np.array(b1*adf.index)

if b1[0] == 1:
    b1f[0] = 1
    b1f = b1f[b1f!=0]
    b1f[0] = 0
else:
    b1f = b1f[b1f!=0]

bdf2 = adf[adf.index.isin(b1f)]
bdf2 = bdf2.reset_index(drop=True)


S1A = []
S1A.append(adf2['county_id'])
S1A.append(adf2['Scaled Population'])
S1A.append(adf2['Dem Seats'])
S1A.append(adf2['Rep Seats'])
S2A = []
S2A.append(bdf2['county_id'])
S2A.append(bdf2['Scaled Population'])
S2A.append(bdf2['Dem Seats'])
S2A.append(bdf2['Rep Seats'])

#M is number of districts for each piece.
M1A = sum(S1A[1]) + sum(S1A[2]) 
#M is number of districts for each piece.
M2A = sum(S2A[1]) + sum(S2A[2]) 
print(sum(adf2['Scaled Population']), sum(bdf2['Scaled Population']))

#starting third:
AA1 = bisect(S1A, 3, 1).X 
AB1 = bisect(S2A, 4, 0).X 



In [ ]:
#side B
A2 = np.array(A2)
a1 = A2
b1 = (1-A2)
a1f = np.array(a1*bdf.index)

if a1[0] == 1:
    a1f[0] = 1
    a1f = a1f[a1f!=0]
    a1f[0] = 0
else:
    a1f = a1f[a1f!=0]

adf3 = bdf[bdf.index.isin(a1f)]
adf3 = adf3.reset_index(drop=True)


b1f = np.array(b1*bdf.index)

if b1[0] == 1:
    b1f[0] = 1
    b1f = b1f[b1f!=0]
    b1f[0] = 0
else:
    b1f = b1f[b1f!=0]

bdf3 = bdf[bdf.index.isin(b1f)]
bdf3 = bdf3.reset_index(drop=True)


S1B = []
S1B.append(adf3['county_id'])
S1B.append(adf3['Scaled Population'])
S1B.append(adf3['Dem Seats'])
S1B.append(adf3['Rep Seats'])
S2B = []
S2B.append(bdf3['county_id'])
S2B.append(bdf3['Scaled Population'])
S2B.append(bdf3['Dem Seats'])
S2B.append(bdf3['Rep Seats'])
#M is number of districts for each piece.
M1B = sum(S1B[1]) + sum(S1B[2]) 
#M is number of districts for each piece.
M2B = sum(S2B[1]) + sum(S2B[2]) 
print(sum(adf2['Scaled Population']), sum(bdf2['Scaled Population']))

#third level

AA2 = bisect(S1B, 3, 1).X 
AB2 = bisect(S2B, 4, 0).X 

In [ ]:
checkSum = 0


In [ ]:
AA1 = np.array(AA1)
a1f = np.array(AA1*adf2.index)

if AA1[0] == 1:
    a1f[0] = 1
    a1f = a1f[a1f!=0]
    a1f[0] = 0
else:
    a1f = a1f[a1f!=0]

D1 = adf2[adf2.index.isin(a1f)]
checkSum+=(sum(D1['Scaled Population']))

AA2 = np.array(AA2)
a1f = np.array(AA2*adf3.index)

if AA2[0] == 1:
    a1f[0] = 1
    a1f = a1f[a1f!=0]
    a1f[0] = 0
else:
    a1f = a1f[a1f!=0]

D2 = adf3[adf3.index.isin(a1f)]

checkSum+=(sum(D2['Scaled Population']))

#remaining 12 districts come from the 6 leftovers being split
#mainly: flipside of AA1 (2), AA2 (2), and AB1(2 and 2), AB2 (2 and 2)

### at this point, we need to make 6 splits. 

In [ ]:
#D3 and 4:
#flipside of AA1:
d3 = 1-AA1
#split

a1f = np.array(d3*adf2.index)

if d3[0] == 1:
    a1f[0] = 1
    a1f = a1f[a1f!=0]
    a1f[0] = 0
else:
    a1f = a1f[a1f!=0]


adf4 = adf2[adf2.index.isin(a1f)]
adf4 = adf4.reset_index(drop=True)


S1A = []
S1A.append(adf4['county_id'])
S1A.append(adf4['Scaled Population'])
S1A.append(adf4['Dem Seats'])
S1A.append(adf4['Rep Seats'])

M1A = sum(S1A[1]) + sum(S1A[2]) 

#starting third:
s3 = bisect(S1A, 2, 1).X 
s3 = np.array(s3)
s4 = 1-s3

a1f = np.array(s3*adf4.index)

if s3[0] == 1:
    a1f[0] = 1
    a1f = a1f[a1f!=0]
    a1f[0] = 0
else:
    a1f = a1f[a1f!=0]

D3 = adf4[adf4.index.isin(a1f)]
checkSum+=(sum(D3['Scaled Population']))

a1f = np.array(s4*adf4.index)

if s4[0] == 1:
    a1f[0] = 1
    a1f = a1f[a1f!=0]
    a1f[0] = 0
else:
    a1f = a1f[a1f!=0]

D4 = adf4[adf4.index.isin(a1f)]

checkSum+=(sum(D4['Scaled Population']))


In [ ]:
#D5 and 6:
#flipside of AA2:
d5 = 1-np.array(AA2)
#split

a1f = np.array(d5*adf3.index)

if d5[0] == 1:
    a1f[0] = 1
    a1f = a1f[a1f!=0]
    a1f[0] = 0
else:
    a1f = a1f[a1f!=0]

adf5 = adf3[adf3.index.isin(a1f)]
adf5 = adf5.reset_index(drop=True)


S1A = []
S1A.append(adf5['county_id'])
S1A.append(adf5['Scaled Population'])
S1A.append(adf5['Dem Seats'])
S1A.append(adf5['Rep Seats'])

M1A = sum(S1A[1]) + sum(S1A[2]) 

#starting third:
s5 = bisect(S1A, 2, 0).X 
s5 = np.array(s5)
s6 = 1-s5

a1f = np.array(s5*adf5.index)

if s5[0] == 1:
    a1f[0] = 1
    a1f = a1f[a1f!=0]
    a1f[0] = 0
else:
    a1f = a1f[a1f!=0]

D5 = adf5[adf5.index.isin(a1f)]
checkSum+=(sum(D5['Scaled Population']))

a1f = np.array(s6*adf5.index)

if s6[0] == 1:
    a1f[0] = 1
    a1f = a1f[a1f!=0]
    a1f[0] = 0
else:
    a1f = a1f[a1f!=0]

D6 = adf5[adf5.index.isin(a1f)]

checkSum+=(sum(D6['Scaled Population']))


In [ ]:
#D7 and 8:
#flipside of AB1[1]:
AB1 = np.array(AB1)
d78 = AB1
d910 = 1 - AB1
#split

a1f = np.array(d78*bdf2.index)

if d78[0] == 1:
    a1f[0] = 1
    a1f = a1f[a1f!=0]
    a1f[0] = 0
else:
    a1f = a1f[a1f!=0]

adf6 = bdf2[bdf2.index.isin(a1f)]
adf6 = adf6.reset_index(drop=True)


S1A = []
S1A.append(adf6['county_id'])
S1A.append(adf6['Scaled Population'])
S1A.append(adf6['Dem Seats'])
S1A.append(adf6['Rep Seats'])

M1A = sum(S1A[1]) + sum(S1A[2])

a1f = np.array(d910*bdf2.index)

if d910[0] == 1:
    a1f[0] = 1
    a1f = a1f[a1f!=0]
    a1f[0] = 0
else:
    a1f = a1f[a1f!=0]

adf7 = bdf2[bdf2.index.isin(a1f)]
adf7 = adf7.reset_index(drop=True)


S2A = []
S2A.append(adf7['county_id'])
S2A.append(adf7['Scaled Population'])
S2A.append(adf7['Dem Seats'])
S2A.append(adf7['Rep Seats'])

M2A = sum(S2A[1]) + sum(S2A[2])

s7 = bisect(S1A, 2, 1).X 
s7 = np.array(s7)
s8 = 1 - s7

s9 = bisect(S2A, 2, 0).X 
s9 = np.array(s9)
s10 = 1-s9


#extract
a1f = np.array(s7*adf6.index)

if s7[0] == 1:
    a1f[0] = 1
    a1f = a1f[a1f!=0]
    a1f[0] = 0
else:
    a1f = a1f[a1f!=0]

D7 = adf6[adf6.index.isin(a1f)]
checkSum+=(sum(D7['Scaled Population']))

a1f = np.array(s8*adf6.index)

if s8[0] == 1:
    a1f[0] = 1
    a1f = a1f[a1f!=0]
    a1f[0] = 0
else:
    a1f = a1f[a1f!=0]

D8 = adf6[adf6.index.isin(a1f)]
checkSum+=(sum(D8['Scaled Population']))

a1f = np.array(s9*adf7.index)

if s9[0] == 1:
    a1f[0] = 1
    a1f = a1f[a1f!=0]
    a1f[0] = 0
else:
    a1f = a1f[a1f!=0]

D9 = adf7[adf7.index.isin(a1f)]
checkSum+=(sum(D9['Scaled Population']))

a1f = np.array(s10*adf7.index)

if s10[0] == 1:
    a1f[0] = 1
    a1f = a1f[a1f!=0]
    a1f[0] = 0
else:
    a1f = a1f[a1f!=0]

D10 = adf7[adf7.index.isin(a1f)]
checkSum+=(sum(D10['Scaled Population']))

In [ ]:
#D11 and 12:
#flipside of AB2[1]:

AB2 = np.array(AB2)
d1112 = AB2
d1314 = 1 - AB2
#split

a1f = np.array(d1112*bdf3.index)

if d1112[0] == 1:
    a1f[0] = 1
    a1f = a1f[a1f!=0]
    a1f[0] = 0
else:
    a1f = a1f[a1f!=0]

adf8 = bdf3[bdf3.index.isin(a1f)]
adf8 = adf8.reset_index(drop=True)


S1A = []
S1A.append(adf8['county_id'])
S1A.append(adf8['Scaled Population'])
S1A.append(adf8['Dem Seats'])
S1A.append(adf8['Rep Seats'])

M1A = sum(S1A[1]) + sum(S1A[2])

a1f = np.array(d1314*bdf3.index)

if d1314[0] == 1:
    a1f[0] = 1
    a1f = a1f[a1f!=0]
    a1f[0] = 0
else:
    a1f = a1f[a1f!=0]

adf9 = bdf3[bdf3.index.isin(a1f)]
adf9 = adf9.reset_index(drop=True)


S2A = []
S2A.append(adf9['county_id'])
S2A.append(adf9['Scaled Population'])
S2A.append(adf9['Dem Seats'])
S2A.append(adf9['Rep Seats'])

M2A = sum(S2A[1]) + sum(S2A[2])

s11 = bisect(S1A, 2, 1).X 
s11 = np.array(s11)
s12 = 1 - s11

s13 = bisect(S2A, 2, 0).X
s13 = np.array(s13)
s14 = 1-s13


#extract
a1f = np.array(s11*adf8.index)

if s11[0] == 1:
    a1f[0] = 1
    a1f = a1f[a1f!=0]
    a1f[0] = 0
else:
    a1f = a1f[a1f!=0]
    
D11 = adf8[adf8.index.isin(a1f)]
checkSum+=(sum(D11['Scaled Population']))

a1f = np.array(s12*adf8.index)

if s12[0] == 1:
    a1f[0] = 1
    a1f = a1f[a1f!=0]
    a1f[0] = 0
else:
    a1f = a1f[a1f!=0]

D12 = adf8[adf8.index.isin(a1f)]
checkSum+=(sum(D12['Scaled Population']))

a1f = np.array(s13*adf9.index)

if s13[0] == 1:
    a1f[0] = 1
    a1f = a1f[a1f!=0]
    a1f[0] = 0
else:
    a1f = a1f[a1f!=0]

D13 = adf9[adf9.index.isin(a1f)]
checkSum+=(sum(D9['Scaled Population']))

a1f = np.array(s14*adf9.index)

if s14[0] == 1:
    a1f[0] = 1
    a1f = a1f[a1f!=0]
    a1f[0] = 0
else:
    a1f = a1f[a1f!=0]

D14 = adf9[adf9.index.isin(a1f)]
checkSum+=(sum(D14['Scaled Population']))


In [ ]:
popsS = []
pops = []
votpops = []
rvotes = []
dvotes = []

D = [D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14]
for d in D:
    popsS.append(sum(d['Scaled Population']))
    pops.append(sum(d['Population']))
    votpops.append(sum(d['Total Votes']))
    rvotes.append(sum(d['Republican Votes']))
    dvotes.append(sum(d['Democratic Votes']))
    
popsS = np.array(popsS)
pops = np.array(pops)
votpops = np.array(votpops)

In [ ]:
np.std(votpops) #New spread

In [ ]:
np.std(votpops/popsS) #New spread

In [ ]:
#comparative spread?
datafile = '/content/drive/My Drive/AI, Games, Markets Project/CurrentDistrictsVoteShares2020.csv'
data2 = pd.read_csv(datafile)

np.std(data2['Total Votes']) #old spread

In [ ]:
dff = []
dis = np.array([])
i = 1
for d in D:
  #print(d['fip'])
  fipcodesd = np.array(d['fip'])
  dff = np.append(dff,fipcodesd)
  disd = np.ones(len(fipcodesd))*i
  dis = np.append(dis,disd)
  i+=1
df = pd.DataFrame()
df['fips'] = dff
df['District'] = 15 - dis #swapping the district numbers around to have the color scale flipped
df['fips'] = df['fips'].map(int)
df['fips'] = df['fips'].map(str)
df['District'] = df['District'].map(int)
#df

### Final Visualization

In [ ]:

fig = px.choropleth_mapbox(df, geojson=counties, locations='fips', color='District',
                           color_continuous_scale=px.colors.sequential.Jet,
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'District':'District'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

# Making map for each district

In [ ]:
fig = px.choropleth_mapbox(df[df['District']==1], geojson=counties, locations='fips',
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'District':'District'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.choropleth_mapbox(df[df['District']==2], geojson=counties, locations='fips',
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'District':'District'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.choropleth_mapbox(df[df['District']==3], geojson=counties, locations='fips',
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'District':'District'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.choropleth_mapbox(df[df['District']==4], geojson=counties, locations='fips',
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'District':'District'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.choropleth_mapbox(df[df['District']==5], geojson=counties, locations='fips',
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'District':'District'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.choropleth_mapbox(df[df['District']==6], geojson=counties, locations='fips',
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'District':'District'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.choropleth_mapbox(df[df['District']==7], geojson=counties, locations='fips',
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'District':'District'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.choropleth_mapbox(df[df['District']==8], geojson=counties, locations='fips',
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'District':'District'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.choropleth_mapbox(df[df['District']==9], geojson=counties, locations='fips',
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'District':'District'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.choropleth_mapbox(df[df['District']==10], geojson=counties, locations='fips',
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'District':'District'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.choropleth_mapbox(df[df['District']==11], geojson=counties, locations='fips',
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'District':'District'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.choropleth_mapbox(df[df['District']==12], geojson=counties, locations='fips',
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'District':'District'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.choropleth_mapbox(df[df['District']==13], geojson=counties, locations='fips',
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'District':'District'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.choropleth_mapbox(df[df['District']==14], geojson=counties, locations='fips',
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'District':'District'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()